# Insall packages

In [ ]:
#installing packages
!pip install tweet-preprocessor
!pip install stanza

     |████████████████████████████████| 286kB 18.4MB/s 


# Import libaries

In [ ]:
import os
import io
import pandas as pd
import numpy as np
import datetime as dt

from google.colab import drive

import re #regex
import preprocessor as p #tweet-preprocessor


#important libraries for preprocessing using NLTK
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('vader_lexicon') # Download the VADER lexicon
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from textblob import TextBlob
import stanza
stanza.download('en')



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
2021-03-17 02:24:58 INFO: Downloading default packages for language: en (English)...
2021-03-17 02:26:18 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read the data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Unstructured/Final_Project/data/vaccine_tweets.csv")

In [ ]:
df

,Unnamed: 0,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,0,https://twitter.com/Volboy51981/status/1237891...,2020-03-11 23:59:51+00:00,@JamesRBrown_3 @Tucker_Passons @wesrucker247 N...,@JamesRBrown_3 @Tucker_Passons @wesrucker247 N...,1237891016566300674,"{'username': 'Volboy51981', 'displayname': 'Vo...",[],[],0,0,0,0,1237841634328342531,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,NaN,NaN,NaN,"[{'username': 'JamesRBrown_3', 'displayname': ..."
1,1,https://twitter.com/shane25873/status/12378909...,2020-03-11 23:59:47+00:00,@stephencapa68 My previous tweet the US are pr...,@stephencapa68 My previous tweet the US are pr...,1237890996848881664,"{'username': 'shane25873', 'displayname': 'Sha...",[],[],1,0,0,0,1237883152493727747,en,"<a href=""http://twitter.com/#!/download/ipad"" ...",http://twitter.com/#!/download/ipad,Twitter for iPad,NaN,NaN,NaN,"[{'username': 'stephencapa68', 'displayname': ..."
2,2,https://twitter.com/nodyejob/status/1237890983...,2020-03-11 23:59:43+00:00,@WillieWholeSlew Not a good scenario. Have you...,@WillieWholeSlew Not a good scenario. Have you...,1237890983104147456,"{'username': 'nodyejob', 'displayname': 'cogdi...",['https://youtu.be/dtKMEAXyPkg'],['https://t.co/wzhVzVKZ21'],0,0,0,0,1237886520482123778,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,NaN,NaN,NaN,NaN
3,3,https://twitter.com/zeusanna/status/1237890981...,2020-03-11 23:59:43+00:00,I have a question. If we can make vaccines for...,I have a question. If we can make vaccines for...,1237890981149564933,"{'username': 'zeusanna', 'displayname': 'zeusa...",[],[],1,0,2,0,1237890981149564933,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN
4,4,https://twitter.com/Vitas72029063/status/12378...,2020-03-11 23:59:39+00:00,@PerthWAustralia You are very optimistic. 100 ...,@PerthWAustralia You are very optimistic. 100 ...,1237890964452044801,"{'username': 'Vitas72029063', 'displayname': '...",[],[],0,0,0,0,1237888254168326146,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,NaN,NaN,NaN,"[{'username': 'PerthWAustralia', 'displayname'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180797,495,https://twitter.com/bond007_crypto/status/1369...,2021-03-10 23:56:23+00:00,@RichMuny @hilaryp60091407 @GovMikeDeWine “All...,@RichMuny @hilaryp60091407 @GovMikeDeWine “All...,1369799326936035333,"{'username': 'bond007_crypto', 'displayname': ...",[],[],1,0,0,0,1369764143931219974,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,"[{'username': 'RichMuny', 'displayname': 'Rich..."
180798,496,https://twitter.com/DavidLonsdale/status/13697...,2021-03-10 23:56:23+00:00,"@tskyliekarma Don't worry, Kylie! I am going t...","@tskyliekarma Don't worry, Kylie! I am going t...",1369799326273318920,"{'username': 'DavidLonsdale', 'displayname': '...",[],[],0,0,0,0,1369763979703226368,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,NaN,NaN,NaN,"[{'username': 'tskyliekarma', 'displayname': '..."
180799,497,https://twitter.com/NCVates/status/13697993225...,2021-03-10 23:56:22+00:00,@JonahDispatch Did Trump discover the formula ...,@JonahDispatch Did Trump discover the formula ...,1369799322548731904,"{'username': 'NCVates', 'displayname': 'NC Vat...",[],[],1,0,2,0,1369794406669225986,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,NaN,NaN,NaN,"[{'username': 'JonahDispatch', 'displayname': ..."
180800,498,https://twitter.com/News

In [ ]:
df.lang.unique() #languages in tweets

array(['en', 'in', 'fr', 'und', 'tl', 'ca', 'et', 'ar', 'ja', 'es', 'it',
       'da', 'ro', 'pt', 'hi', 'zh', 'nl', 'bn', 'de', 'tr', 'ta', 'ht',
       'ne', 'cs', 'mr', 'sl', 'sv', 'vi', 'ko', 'ur', 'th', 'fi', 'el',
       'km', 'pl', 'cy', 'hu', 'lt', 'te', 'no', 'or', 'eu', 'lv', 'fa',
       'sr', 'pa'], dtype=object)

## Select English tweets and useful columns

In [ ]:
data = df[df['lang'] =='en']  #select the english tweets only 
data.reset_index(inplace=True) #reset index

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175873 entries, 0 to 175872
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   index            175873 non-null  int64  
 1   Unnamed: 0       175873 non-null  int64  
 2   url              175873 non-null  object 
 3   date             175873 non-null  object 
 4   content          175873 non-null  object 
 5   renderedContent  175873 non-null  object 
 6   id               175873 non-null  int64  
 7   user             175873 non-null  object 
 8   outlinks         175873 non-null  object 
 9   tcooutlinks      175873 non-null  object 
 10  replyCount       175873 non-null  int64  
 11  retweetCount     175873 non-null  int64  
 12  likeCount        175873 non-null  int64  
 13  quoteCount       175873 non-null  int64  
 14  conversationId   175873 non-null  int64  
 15  lang             175873 non-null  object 
 16  source           175787 non-null  obje

In [ ]:
columns_keep = ["date","content","id", "user","replyCount", "retweetCount", "likeCount", "quoteCount", "sourceLabel"]

In [ ]:
data = data[columns_keep]
data

,date,content,id,user,replyCount,retweetCount,likeCount,quoteCount,sourceLabel
0,2020-03-11 23:59:51+00:00,@JamesRBrown_3 @Tucker_Passons @wesrucker247 N...,1237891016566300674,"{'username': 'Volboy51981', 'displayname': 'Vo...",0,0,0,0,Twitter for Android
1,2020-03-11 23:59:47+00:00,@stephencapa68 My previous tweet the US are pr...,1237890996848881664,"{'username': 'shane25873', 'displayname': 'Sha...",1,0,0,0,Twitter for iPad
2,2020-03-11 23:59:43+00:00,@WillieWholeSlew Not a good scenario. Have you...,1237890983104147456,"{'username': 'nodyejob', 'displayname': 'cogdi...",0,0,0,0,Twitter Web App
3,2020-03-11 23:59:43+00:00,I have a question. If we can make vaccines for...,1237890981149564933,"{'username': 'zeusanna', 'displayname': 'zeusa...",1,0,2,0,Twitter for iPhone
4,2020-03-11 23:59:39+00:00,@PerthWAustralia You are very optimistic. 100 ...,1237890964452044801,"{'username': 'Vitas72029063', 'displayname': '...",0,0,0,0,Twitter for Android
...,...,...,...,...,...,...,...,...,...
175868,2021-03-10 23:56:23+00:00,@RichMuny @hilaryp60091407 @GovMikeDeWine “All...,1369799326936035333,"{'username': 'bond007_crypto', 'displayname': ...",1,0,0,0,Twitter for iPhone
175869,2021-03-10 23:56:23+00:00,"@tskyliekarma Don't worry, Kylie! I am going t...",1369799326273318920,"{'username': 'DavidLonsdale', 'displayname': '...",0,0,0,0,Twitter for Android
175870,2021-03-10 23:56:22+00:00,@JonahDispatch Did Trump discover the formula ...,1369799322548731904,"{'username': 'NCVates', 'displayname': 'NC Vat...",1,0,2,0,Twitter Web App
175871,2021-03-10 23:56:22+00:00,Black and Hispanic Communities Grapple With Va...,1369799320095121411,"{'username': 'NewsChantUSA', 'displayname': 'N...",0,0,0,0,WordPress.com


# Clean the tweets

## Remove URLs, Mentions

In [ ]:
data['text_cleaned'] = data['content'].apply(lambda x: p.clean(x))

In [ ]:
data

,date,content,id,user,replyCount,retweetCount,likeCount,quoteCount,sourceLabel,text_cleaned
0,2020-03-11 23:59:51+00:00,@JamesRBrown_3 @Tucker_Passons @wesrucker247 N...,1237891016566300674,"{'username': 'Volboy51981', 'displayname': 'Vo...",0,0,0,0,Twitter for Android,No vaccine for the flu either.
1,2020-03-11 23:59:47+00:00,@stephencapa68 My previous tweet the US are pr...,1237890996848881664,"{'username': 'shane25873', 'displayname': 'Sha...",1,0,0,0,Twitter for iPad,My previous tweet the US are predicting /3 of ...
2,2020-03-11 23:59:43+00:00,@WillieWholeSlew Not a good scenario. Have you...,1237890983104147456,"{'username': 'nodyejob', 'displayname': 'cogdi...",0,0,0,0,Twitter Web App,Not a good scenario. Have you ever seen I Am L...
3,2020-03-11 23:59:43+00:00,I have a question. If we can make vaccines for...,1237890981149564933,"{'username': 'zeusanna', 'displayname': 'zeusa...",1,0,2,0,Twitter for iPhone,I have a question. If we can make vaccines for...
4,2020-03-11 23:59:39+00:00,@PerthWAustralia You are very optimistic. 100 ...,1237890964452044801,"{'username': 'Vitas72029063', 'displayname': '...",0,0,0,0,Twitter for Android,You are very optimistic. million people will b...
...,...,...,...,...,...,...,...,...,...,...
175868,2021-03-10 23:56:23+00:00,@RichMuny @hilaryp60091407 @GovMikeDeWine “All...,1369799326936035333,"{'username': 'bond007_crypto', 'displayname': ...",1,0,0,0,Twitter for iPhone,All I did was obey and wear a mask. All I did ...
175869,2021-03-10 23:56:23+00:00,"@tskyliekarma Don't worry, Kylie! I am going t...",1369799326273318920,"{'username': 'DavidLonsdale', 'displayname': '...",0,0,0,0,Twitter for Android,"Don't worry, Kylie! I am going to book an appo..."
175870,2021-03-10 23:56:22+00:00,@JonahDispatch Did Trump discover the formula ...,1369799322548731904,"{'username': 'NCVates', 'displayname': 'NC Vat...",1,0,2,0,Twitter Web App,Did Trump discover the formula for the vaccine...
175871,2021-03-10 23:56:22+00:00,Black and Hispanic Communities Grapple With Va...,1369799320095121411,"{'username': 'NewsChantUSA', 'displayname': 'N...",0,0,0,0,WordPress.com,Black and Hispanic Communities Grapple With Va...


## Remove digits, non-punctuation signs


In [ ]:
data['text_cleaned']=data['text_cleaned'].str.replace('\d+', '')
data['text_cleaned']=data['text_cleaned'].str.replace('[^\w\s\.\,\?\!]','')


In [ ]:
data['date']=pd.to_datetime(data['date'], format='%Y-%m-%d').dt.date #change datatime to date

In [ ]:
data

,date,content,id,user,replyCount,retweetCount,likeCount,quoteCount,sourceLabel,text_cleaned
0,2020-03-11,@JamesRBrown_3 @Tucker_Passons @wesrucker247 N...,1237891016566300674,"{'username': 'Volboy51981', 'displayname': 'Vo...",0,0,0,0,Twitter for Android,No vaccine for the flu either.
1,2020-03-11,@stephencapa68 My previous tweet the US are pr...,1237890996848881664,"{'username': 'shane25873', 'displayname': 'Sha...",1,0,0,0,Twitter for iPad,My previous tweet the US are predicting of th...
2,2020-03-11,@WillieWholeSlew Not a good scenario. Have you...,1237890983104147456,"{'username': 'nodyejob', 'displayname': 'cogdi...",0,0,0,0,Twitter Web App,Not a good scenario. Have you ever seen I Am L...
3,2020-03-11,I have a question. If we can make vaccines for...,1237890981149564933,"{'username': 'zeusanna', 'displayname': 'zeusa...",1,0,2,0,Twitter for iPhone,I have a question. If we can make vaccines for...
4,2020-03-11,@PerthWAustralia You are very optimistic. 100 ...,1237890964452044801,"{'username': 'Vitas72029063', 'displayname': '...",0,0,0,0,Twitter for Android,You are very optimistic. million people will b...
...,...,...,...,...,...,...,...,...,...,...
175868,2021-03-10,@RichMuny @hilaryp60091407 @GovMikeDeWine “All...,1369799326936035333,"{'username': 'bond007_crypto', 'displayname': ...",1,0,0,0,Twitter for iPhone,All I did was obey and wear a mask. All I did ...
175869,2021-03-10,"@tskyliekarma Don't worry, Kylie! I am going t...",1369799326273318920,"{'username': 'DavidLonsdale', 'displayname': '...",0,0,0,0,Twitter for Android,"Dont worry, Kylie! I am going to book an appoi..."
175870,2021-03-10,@JonahDispatch Did Trump discover the formula ...,1369799322548731904,"{'username': 'NCVates', 'displayname': 'NC Vat...",1,0,2,0,Twitter Web App,Did Trump discover the formula for the vaccine...
175871,2021-03-10,Black and Hispanic Communities Grapple With Va...,1369799320095121411,"{'username': 'NewsChantUSA', 'displayname': 'N...",0,0,0,0,WordPress.com,Black and Hispanic Communities Grapple With Va...


In [ ]:
#check duplicates
duplicates = data["text_cleaned"].duplicated() 
data[duplicates]

,date,content,id,user,replyCount,retweetCount,likeCount,quoteCount,sourceLabel,text_cleaned
23,2020-03-11,@BorisJohnson In the enquiry after 1000's of a...,1237890722969387008,"{'username': 'Mr__XYZ', 'displayname': '🇬🇧 Nig...",0,0,0,0,Twitter Web App,In the enquiry after s of avoidable deaths due...
28,2020-03-11,@BorisJohnson In the enquiry after 1000's of a...,1237890658670497793,"{'username': 'Mr__XYZ', 'displayname': '🇬🇧 Nig...",0,0,0,0,Twitter Web App,In the enquiry after s of avoidable deaths due...
34,2020-03-11,@BorisJohnson In the enquiry after 1000's of a...,1237890610557673475,"{'username': 'Mr__XYZ', 'displayname': '🇬🇧 Nig...",0,0,0,0,Twitter Web App,In the enquiry after s of avoidable deaths due...
80,2020-03-11,@PHE_uk @MattHancock In the enquiry after 1000...,1237890149733687301,"{'username': 'Mr__XYZ', 'displayname': '🇬🇧 Nig...",0,0,0,0,Twitter Web App,In the enquiry after s of avoidable deaths due...
108,2020-03-11,@NHSEngland @MattHancock In the enquiry after ...,1237889831104995330,"{'username': 'Mr__XYZ', 'displayname': '🇬🇧 Nig...",0,0,0,0,Twitter Web App,In the enquiry after s of avoidable deaths due...
...,...,...,...,...,...,...,...,...,...,...
175575,2021-03-10,@CBSNews @edokeefe USA is buying up their home...,1369799859499335685,"{'username': 'ssihusa', 'displayname': 'Alexan...",0,0,0,0,Twitter for Android,USA is buying up their home made vaccine that ...
175655,2021-03-10,Leaked documents reveal major concerns over qu...,1369799746748026880,"{'username': 'DiazHub', 'displayname': 'Diazhu...",0,0,0,0,WordPress.com,Leaked documents reveal major concerns over qu...
175658,2021-03-10,Does Merck's 'Vaccine Hesitancy' Warrant a Clo...,1369799743082270723,"{'username': 'crwriter1', 'displayname': 'Joyc...",0,0,1,0,myTweetPack,Does Mercks Vaccine Hesitancy Warrant a Closer...
175785,2021-03-10,Biden boosts US vaccine stockpile as world wai...,1369799489511436288,"{'username': 'KCRG', 'displayname': 'KCRG-TV9'...",5,0,8,0,SocialNewsDesk,Biden boosts US vaccine stockpile as world waits


In [ ]:
#delete duplicated cases
data = data[~duplicates]

In [ ]:
data.set_index("date", inplace=True)

In [ ]:
data

,content,id,user,replyCount,retweetCount,likeCount,quoteCount,sourceLabel,text_cleaned
date,,,,,,,,,
2020-03-11,@JamesRBrown_3 @Tucker_Passons @wesrucker247 N...,1237891016566300674,"{'username': 'Volboy51981', 'displayname': 'Vo...",0,0,0,0,Twitter for Android,No vaccine for the flu either.
2020-03-11,@stephencapa68 My previous tweet the US are pr...,1237890996848881664,"{'username': 'shane25873', 'displayname': 'Sha...",1,0,0,0,Twitter for iPad,My previous tweet the US are predicting of th...
2020-03-11,@WillieWholeSlew Not a good scenario. Have you...,1237890983104147456,"{'username': 'nodyejob', 'displayname': 'cogdi...",0,0,0,0,Twitter Web App,Not a good scenario. Have you ever seen I Am L...
2020-03-11,I have a question. If we can make vaccines for...,1237890981149564933,"{'username': 'zeusanna', 'displayname': 'zeusa...",1,0,2,0,Twitter for iPhone,I have a question. If we can make vaccines for...
2020-03-11,@PerthWAustralia You are very optimistic. 100 ...,1237890964452044801,"{'username': 'Vitas72029063', 'displayname': '...",0,0,0,0,Twitter for Android,You are very optimistic. million people will b...
...,...,...,...,...,...,...,...,...,...
2021-03-10,@RichMuny @hilaryp60091407 @GovMikeDeWine “All...,1369799326936035333,"{'username': 'bond007_crypto', 'displayname': ...",1,0,0,0,Twitter for iPhone,All I did was obey and wear a mask. All I did ...
2021-03-10,"@tskyliekarma Don't worry, Kylie! I am going t...",1369799326273318920,"{'username': 'DavidLonsdale', 'displayname': '...",0,0,0,0,Twitter for Android,"Dont worry, Kylie! I am going to book an appoi..."
2021-03-10,@JonahDispatch Did Trump discover the formula ...,1369799322548731904,"{'username': 'NCVates', 'displayname': 'NC Vat...",1,0,2,0,Twitter Web App,Did Trump discover the formula for the vaccine...


In [ ]:
data.index.min(), data.index.max()

(datetime.date(2020, 3, 11), datetime.date(2021, 3, 10))

# Sentiment Analysis with NLTK Vader

VADER stands from Valence Aware Dictionary and sEntiment Reasoner and is specifically attuned to sentiments expressed in social media.

* positive: compound score>=0.05
* neutral: compound score between -0.05 and 0.05
* negative: compound score<=-0.05





In [ ]:
# Initialize sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Obtaining NLTK scores
data['nltk_scores'] = data['text_cleaned'].apply(lambda x: sia.polarity_scores(x))

# Obtaining NLTK compound score
data['nltk_cmp_score'] = data['nltk_scores'].apply(lambda score_dict: score_dict['compound'])

# Set threshold to define neutral sentiment
neutral_thresh = 0.05

# Categorize scores into the sentiments of positive, neutral or negative
data['nltk_sentiment'] = data['nltk_cmp_score'].apply(lambda c: 'Positive' if c >= neutral_thresh else ('Negative' if c <= -(neutral_thresh) else 'Neutral'))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [ ]:
pd.set_option('display.max_colwidth', None)

data[['text_cleaned', 'nltk_scores','nltk_cmp_score', 'nltk_sentiment']]

,text_cleaned,nltk_scores,nltk_cmp_score,nltk_sentiment
date,,,,
2020-03-11,No vaccine for the flu either.,"{'neg': 0.545, 'neu': 0.455, 'pos': 0.0, 'compound': -0.5859}",-0.5859,Negative
2020-03-11,"My previous tweet the US are predicting of their nation will get it.Theyre having great difficulty developing a vaccine, as its evolving too quick for them.It must be wonderful knowing youll be fine, so fuck the rest of us!Im working with my docs to keep me informed.","{'neg': 0.119, 'neu': 0.71, 'pos': 0.171, 'compound': 0.459}",0.4590,Positive
2020-03-11,"Not a good scenario. Have you ever seen I Am Legend with Will Smith? A miracle cure all vaccine ultimately turned people into violen, lightningfast zombies.","{'neg': 0.088, 'neu': 0.772, 'pos': 0.14, 'compound': 0.3387}",0.3387,Positive
2020-03-11,"I have a question. If we can make vaccines for coronavirus why dont we have one for the common cold, which is a coronavirus?","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,Neutral
2020-03-11,"You are very optimistic. million people will be infected at its peak at the same time, not million in total. Many people will be infected more than once or twice. dead recovered. will be infected more than once. months till vaccine is ready","{'neg': 0.248, 'neu': 0.661, 'pos': 0.091, 'compound': -0.8692}",-0.8692,Negative
...,...,...,...,...
2021-03-10,All I did was obey and wear a mask. All I did was obey and take the experimental vaccine.....sheep!,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,Neutral
2021-03-10,"Dont worry, Kylie! I am going to book an appointment as soon as possible after the local pharmacies here in Toronto get the vaccines. I will be happy to answer any questions after I get my shot! Okay?","{'neg': 0.0, 'neu': 0.788, 'pos': 0.212, 'compound': 0.822}",0.8220,Positive
2021-03-10,Did Trump discover the formula for the vaccine while in the bunker at the th hole at Doral?,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,Neutral


# Sentiment Analysis with TextBlob

Similar to NLTK Vader, the TextBlob sentiment classifier is based on a bag of words approach. In fact, TextBlob is built upon the NLTK and pattern libraries.

In [ ]:
# Obtain polarity scores generated by TextBlob
data['textblob_score'] = data['text_cleaned'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Set threshold to define neutral sentiment
neutral_thresh = 0.05

# Convert polarity score into sentiment categories
data['textblob_sentiment'] = data['textblob_score'].apply(lambda c: 'Positive' if c >= neutral_thresh else ('Negative' if c <= -(neutral_thresh) else 'Neutral'))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data[['text_cleaned', 'nltk_cmp_score', 'nltk_sentiment','textblob_score','textblob_sentiment']]

,text_cleaned,nltk_cmp_score,nltk_sentiment,textblob_score,textblob_sentiment
date,,,,,
2020-03-11,No vaccine for the flu either.,-0.5859,Negative,0.000000,Neutral
2020-03-11,"My previous tweet the US are predicting of their nation will get it.Theyre having great difficulty developing a vaccine, as its evolving too quick for them.It must be wonderful knowing youll be fine, so fuck the rest of us!Im working with my docs to keep me informed.",0.4590,Positive,0.330556,Positive
2020-03-11,"Not a good scenario. Have you ever seen I Am Legend with Will Smith? A miracle cure all vaccine ultimately turned people into violen, lightningfast zombies.",0.3387,Positive,-0.175000,Negative
2020-03-11,"I have a question. If we can make vaccines for coronavirus why dont we have one for the common cold, which is a coronavirus?",0.0000,Neutral,-0.450000,Negative
2020-03-11,"You are very optimistic. million people will be infected at its peak at the same time, not million in total. Many people will be infected more than once or twice. dead recovered. will be infected more than once. months till vaccine is ready",-0.8692,Negative,0.212500,Positive
...,...,...,...,...,...
2021-03-10,All I did was obey and wear a mask. All I did was obey and take the experimental vaccine.....sheep!,0.0000,Neutral,0.125000,Positive
2021-03-10,"Dont worry, Kylie! I am going to book an appointment as soon as possible after the local pharmacies here in Toronto get the vaccines. I will be happy to answer any questions after I get my shot! Okay?",0.8220,Positive,0.375000,Positive
2021-03-10,Did Trump discover the formula for the vaccine while in the bunker at the th hole at Doral?,0.0000,Neutral,0.000000,Neutral


# Sentiment Analysis with Stanza

Stanza is the official Python NLP library of Stanford NLP Group, replacing the Java-based CoreNLP. The modules are built on top of PyTorch, and its pre-built sentiment analyzer is based on a CNN classifier trained on datasets including Stanford Sentiment Treeback and Airline Twitter Sentiment.

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment')

# Obtain (average) sentiment score generated by Stanza for each Tweet
def stanza_analyze(Text):
    document = nlp(Text)
    return np.mean([(i.sentiment - 1) for i in document.sentences]) # Minus 1 so as to bring score range of [0,2] to [-1,1]

# Obtain sentiment categorical score generated by Stanza
data['stanza_score'] = data['text_cleaned'].apply(lambda x: stanza_analyze(x))

neutral_thresh = 0.05

# Convert average Stanza sentiment score into sentiment categories
data['stanza_sentiment'] = data['stanza_score'].apply(lambda c: 'Positive' if c >= neutral_thresh else ('Negative' if c <= -(neutral_thresh) else 'Neutral'))


2021-03-17 02:28:34 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

2021-03-17 02:28:35 INFO: Use device: gpu
2021-03-17 02:28:35 INFO: Loading: tokenize
2021-03-17 02:28:45 INFO: Loading: sentiment
2021-03-17 02:28:46 INFO: Done loading processors!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [ ]:
data[['text_cleaned', 'nltk_cmp_score', 'nltk_sentiment','textblob_score','textblob_sentiment','stanza_score','stanza_sentiment']]

,text_cleaned,nltk_cmp_score,nltk_sentiment,textblob_score,textblob_sentiment,stanza_score,stanza_sentiment
date,,,,,,,
2020-03-11,No vaccine for the flu either.,-0.5859,Negative,0.000000,Neutral,-1.000000,Negative
2020-03-11,"My previous tweet the US are predicting of their nation will get it.Theyre having great difficulty developing a vaccine, as its evolving too quick for them.It must be wonderful knowing youll be fine, so fuck the rest of us!Im working with my docs to keep me informed.",0.4590,Positive,0.330556,Positive,-0.500000,Negative
2020-03-11,"Not a good scenario. Have you ever seen I Am Legend with Will Smith? A miracle cure all vaccine ultimately turned people into violen, lightningfast zombies.",0.3387,Positive,-0.175000,Negative,-0.666667,Negative
2020-03-11,"I have a question. If we can make vaccines for coronavirus why dont we have one for the common cold, which is a coronavirus?",0.0000,Neutral,-0.450000,Negative,-0.500000,Negative
2020-03-11,"You are very optimistic. million people will be infected at its peak at the same time, not million in total. Many people will be infected more than once or twice. dead recovered. will be infected more than once. months till vaccine is ready",-0.8692,Negative,0.212500,Positive,0.000000,Neutral
...,...,...,...,...,...,...,...
2021-03-10,All I did was obey and wear a mask. All I did was obey and take the experimental vaccine.....sheep!,0.0000,Neutral,0.125000,Positive,-0.500000,Negative
2021-03-10,"Dont worry, Kylie! I am going to book an appointment as soon as possible after the local pharmacies here in Toronto get the vaccines. I will be happy to answer any questions after I get my shot! Okay?",0.8220,Positive,0.375000,Positive,0.500000,Positive
2021-03-10,Did Trump discover the formula for the vaccine while in the bunker at the th hole at Doral?,0.0000,Neutral,0.000000,Neutral,0.000000,Neutral


# Create a composite score based on the average of three

In [ ]:
# create a composite score based on three scores
data['composite_score'] = (data['nltk_cmp_score'] + data ['textblob_score'] + data['stanza_score'])/3

neutral_thresh = 0.05

# Convert composite score into sentiment categories
data['composite_sentiment'] = data['composite_score'].apply(lambda c: 'Positive' if c >= neutral_thresh else ('Negative' if c <= -(neutral_thresh) else 'Neutral'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
data[['text_cleaned', 'nltk_cmp_score', 'nltk_sentiment','textblob_score','textblob_sentiment','stanza_score','stanza_sentiment','composite_score','composite_sentiment']]

,text_cleaned,nltk_cmp_score,nltk_sentiment,textblob_score,textblob_sentiment,stanza_score,stanza_sentiment,composite_score,composite_sentiment
date,,,,,,,,,
2020-03-11,No vaccine for the flu either.,-0.5859,Negative,0.000000,Neutral,-1.000000,Negative,-0.528633,Negative
2020-03-11,"My previous tweet the US are predicting of their nation will get it.Theyre having great difficulty developing a vaccine, as its evolving too quick for them.It must be wonderful knowing youll be fine, so fuck the rest of us!Im working with my docs to keep me informed.",0.4590,Positive,0.330556,Positive,-0.500000,Negative,0.096519,Positive
2020-03-11,"Not a good scenario. Have you ever seen I Am Legend with Will Smith? A miracle cure all vaccine ultimately turned people into violen, lightningfast zombies.",0.3387,Positive,-0.175000,Negative,-0.666667,Negative,-0.167656,Negative
2020-03-11,"I have a question. If we can make vaccines for coronavirus why dont we have one for the common cold, which is a coronavirus?",0.0000,Neutral,-0.450000,Negative,-0.500000,Negative,-0.316667,Negative
2020-03-11,"You are very optimistic. million people will be infected at its peak at the same time, not million in total. Many people will be infected more than once or twice. dead recovered. will be infected more than once. months till vaccine is ready",-0.8692,Negative,0.212500,Positive,0.000000,Neutral,-0.218900,Negative
...,...,...,...,...,...,...,...,...,...
2021-03-10,All I did was obey and wear a mask. All I did was obey and take the experimental vaccine.....sheep!,0.0000,Neutral,0.125000,Positive,-0.500000,Negative,-0.125000,Negative
2021-03-10,"Dont worry, Kylie! I am going to book an appointment as soon as possible after the local pharmacies here in Toronto get the vaccines. I will be happy to answer any questions after I get my shot! Okay?",0.8220,Positive,0.375000,Positive,0.500000,Positive,0.565667,Positive
2021-03-10,Did Trump discover the formula for the vaccine while in the bunker at the th hole at Doral?,0.0000,Neutral,0.000000,Neutral,0.000000,Neutral,0.000000,Neutral


In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168742 entries, 0 to 168741
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   date                 168742 non-null  object 
 1   content              168742 non-null  object 
 2   id                   168742 non-null  int64  
 3   user                 168742 non-null  object 
 4   replyCount           168742 non-null  int64  
 5   retweetCount         168742 non-null  int64  
 6   likeCount            168742 non-null  int64  
 7   quoteCount           168742 non-null  int64  
 8   sourceLabel          168658 non-null  object 
 9   text_cleaned         168742 non-null  object 
 10  nltk_scores          168742 non-null  object 
 11  nltk_cmp_score       168742 non-null  float64
 12  nltk_sentiment       168742 non-null  object 
 13  textblob_score       168742 non-null  float64
 14  textblob_sentiment   168742 non-null  object 
 15  stanza_score     

In [ ]:
tweet_sentiment_df = data.drop(columns=['content','user'])

In [ ]:
tweet_sentiment_df.to_csv("/content/drive/MyDrive/Unstructured/Final_Project/data/tweet_sentiments.csv")

Reference: 

1.   [Leung, Kenneth. Jan 26, 2021. "COVID-19 Vaccine — What’s the Public Sentiment?", *Towards Data Science* ](https://towardsdatascience.com/covid-19-vaccine-whats-the-public-sentiment-7149c9b42b99)
2.   [Qi, P., Zhang, Y., Zhang, Y., Bolton, J. and Manning, C.D., 2020. Stanza: A Python natural language processing toolkit for many human languages. arXiv preprint arXiv:2003.07082.](https://stanfordnlp.github.io/stanza/index.html)


